In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# Define path to dataset directory
dataset_dir = "../DataPreprocessing/FinalDataset/"

# Define image size for resizing
image_size = (299, 299)  # Use 299x299 for InceptionV3

# Define the number of classes
num_classes = len(os.listdir(dataset_dir))

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Rescale pixel values to [0, 1]
    rotation_range=30,  # Increased rotation range
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # Added shear range
    zoom_range=0.2,  # Added zoom range
    horizontal_flip=True,
    validation_split=0.2
)

# Load the pre-trained model (e.g., InceptionV3)
base_model = keras.applications.InceptionV3(
    weights="imagenet",
    include_top=False,
    input_shape=(*image_size, 3)
)

# Add custom layers for classification with dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)  # Reduced dense layer size
x = Dropout(0.5)(x)  # Increased dropout rate
predictions = Dense(num_classes, activation="softmax")(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

# Print a summary of the model architecture
model.summary()

# Create data generators for training and validation
batch_size = 32
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)
valid_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Define a model checkpoint to save the best model in ".h5" format
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)

# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model with increased epochs
epochs = 20
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_generator,
    callbacks=[checkpoint, early_stopping]  # Save the best model and early stopping
)

# Load the best model for evaluation
best_model = keras.models.load_model("best_model.h5")

# Evaluate the model on the validation dataset
valid_generator.reset()
valid_loss, valid_accuracy = best_model.evaluate(valid_generator)

# Evaluate the model on the test dataset
test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)
test_loss, test_accuracy = best_model.evaluate(test_generator)

print(f"Validation Loss: {valid_loss:.4f}")
print(f"Validation Accuracy: {valid_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
